In [ ]:
import trimesh
import open3d as o3d
import correct_lidar
import numpy as np

Open clear day scan (2022-12-12 0800, which is actually at 0100)

In [ ]:
path = "/Users/elischwat/Development/data/sublimationofsnow/lidar_raw/l1/2022-12-12/2022-12-12_08-00-15.lvx"

In [ ]:
# Create ply version
points = correct_lidar.load_first_file('l1', file_search=path)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points.T)
o3d.io.write_point_cloud(
    filename=path.replace(".lvx", ".ply"), 
    pointcloud=pcd
)

In [ ]:
path = "/Users/elischwat/Development/data/sublimationofsnow/lidar_raw/l1/2022-12-12/2022-12-12_08-00-15.ply"

In [ ]:
pcd = o3d.io.read_point_cloud(path)
o3d.visualization.draw_geometries([pcd])

In [ ]:
points_array = np.asarray(pcd.points)

Test using meshes for two points, and union them

In [ ]:
sphere1 = trimesh.primitives.Sphere(radius = 0.2, center=points_array[0], subdivisions = 1)
sphere2 = trimesh.primitives.Sphere(radius = 0.2, center=points_array[1], subdivisions = 1)
union_sphere = sphere1.union(sphere2, check_volume = False)

union_sphere.show()

Test the contains function

In [ ]:
union_sphere.contains(points_array[:5])

Now do the same for lots of points

In [ ]:
blob = trimesh.primitives.Sphere(radius = 0.25, center=points_array[0], subdivisions = 1)
for pt in points_array[1:1000]:
    new_sphere = trimesh.primitives.Sphere(radius = 0.25, center=pt, subdivisions = 1)
    blob = blob.union(new_sphere, check_volume = False)

That was slow. Lets split it into two blobs, and merge those

In [ ]:
blob = trimesh.primitives.Sphere(radius = 0.25, center=points_array[0], subdivisions = 1)
for pt in points_array[1:500]:
    new_sphere = trimesh.primitives.Sphere(radius = 0.25, center=pt, subdivisions = 1)
    blob = blob.union(new_sphere, check_volume = False)

In [ ]:
blob2 = trimesh.primitives.Sphere(radius = 0.25, center=points_array[500], subdivisions = 1)
for pt in points_array[500:1000]:
    new_sphere = trimesh.primitives.Sphere(radius = 0.25, center=pt, subdivisions = 1)
    blob2 = blob2.union(new_sphere, check_volume = False)

In [ ]:
blob.union(blob2, check_volume = False)

That's a good bit faster... do i need to implement my own "cascaded union"

Let's just brute force a huge batch and see what happens.

In [ ]:
blob = trimesh.primitives.Sphere(radius = 0.25, center=points_array[0], subdivisions = 1)
for pt in points_array[1:]:
    new_sphere = trimesh.primitives.Sphere(radius = 0.25, center=pt, subdivisions = 1)
    blob = blob.union(new_sphere, check_volume = False)

Use this to simplify the geometry

In [ ]:
blob.simplify_quadric_decimation(1000).show()